In [171]:
from bs4 import BeautifulSoup
import requests
import re
import json
import pandas as pd
from urllib.parse import urlencode

In [141]:
# URL = 'https://www.sephora.com//product/the-ordinary-deciem-hyaluronic-acid-2-b5-P427419?skuId=2031375&icid2=products'
# #URL = 'https://www.sephora.com/product/sephora-collection-rose-quartz-gua-sha-P502996?skuId=2583318&icid2=products%20grid:p502996:product'

# headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"}

# page = requests.get(URL, headers=headers)

# soup1 = BeautifulSoup(page.content, "html.parser")

# soup2 = BeautifulSoup(soup1.prettify(), "html.parser")

# testing = json.loads(soup2.find("script", {"id": "linkStore"}).text)

# print(testing)

# print(testing['page']["product"]["productDetails"].get('productId', None))


In [142]:
def get_product_breadcrumbs(json_dict):
    product_info = json_dict["page"]["product"]
    return list(map(lambda x: x["item"]["name"], json.loads(product_info["breadcrumbsSeoJsonLd"])["itemListElement"]))



In [143]:
def get_brand(json_dict):
    return json_dict["page"]["product"]["productDetails"]['brand'].get('displayName', None)

In [144]:
def get_product_name(json_dict):
    return json_dict["page"]["product"]["productDetails"].get('displayName', None)

In [145]:
def get_rating(json_dict):
    return json_dict["page"]["product"]["productDetails"].get('rating', None)

In [146]:
def get_lovesCount(json_dict):
    return json_dict["page"]["product"]["productDetails"].get('lovesCount', None)

In [147]:
def get_review_num(json_dict):
    return json_dict["page"]["product"]["productDetails"].get('reviews', None)

In [148]:
def get_skin_concern(json_dict):
    skin_concern_results = json_dict["page"]["biOptions"].get("skinConcerns", None)
    skin_concern_list = []
    if skin_concern_results!= None:
        for dicts in skin_concern_results:
            skin_concern_list.append(dicts['value'])
    return skin_concern_list

In [149]:
def get_skin_type(json_dict):
    skin_type_results = json_dict["page"]["biOptions"].get("skinType", None)
    skin_type_list = []
    if skin_type_results!= None:
        for dicts in skin_type_results:

            skin_type_list.append(dicts['displayName'])
    return skin_type_list


In [150]:
def get_sku(json_dict):
    return json_dict["page"]["product"]["currentSku"].get("skuId", None)

In [151]:
def get_size(json_dict):
    return json_dict["page"]["product"]["currentSku"].get("size", None)

In [152]:
def get_price(json_dict):
    price = json_dict["page"]["product"]["currentSku"].get("listPrice", None)
    if price!= None:
        price = price[1:]
    return price


In [153]:
def get_child_sku(json_dict):
    find_child = json_dict["page"]["product"].get("regularChildSkus", None)
    current_sku = json_dict["page"]["product"]["currentSku"]["skuId"]
    child_products = []
    if find_child is not None:
        for item in find_child:
            if item['skuId']!=current_sku:
                child_products.append(item['skuId'])
    return child_products


In [154]:
def get_item_id(json_dict):
    return json_dict['page']["product"]["productDetails"].get('productId', None)

In [155]:
def get_similar_products(product_id, headers):
    similar_product_url = 'https://sephora.cnstrc.com/recommendations/v1/pods/similar-products-test?c=ciojs-client-2.41.0&key=u7PNVQx-prod-en-us&i=0c7be738-3d64-4f03-b5fd-7c5df9afefdc&s=23&num_results=5&item_id='+product_id
    similar_page = requests.get(similar_product_url, headers=headers)
    similar_page = json.loads(similar_page.text)
    
    products_list = []
    if similar_page["response"].get("results", None)!= None:
        for item in similar_page["response"]["results"]:
            products_list.append(item["data"]["currentSku"].get("skuId", None))
    else:
        products_list.append('')

    return products_list

In [206]:
#loop through page 1-49 and loop through every product to get the url
def get_skincare_info(url, total_pg, headers, api):
    skincare_dict ={}
    for pg in range(1, total_pg+1):
        complete_url = url + str(pg)
        print(complete_url)
        params = {'api_key':api, 'url':complete_url}
        page = requests.get('http://api.scraperapi.com', params =urlencode(params), headers=headers)
        #page = requests.get(complete_url, headers=headers)
        #print(page)
        soup = BeautifulSoup(page.content, "html.parser")

        #print(soup.prettify())
        link_lst = list(map(lambda x: 'https://www.sephora.com/'+x["href"], soup.find_all('a', {'class':'css-klx76'}, href=True)))
        print(link_lst)
        print("first element")
        print(link_lst[0])
        for link in link_lst[:2]:
            #add api here as well
            #product_pg = requests.get(link,headers=headers)
            #link = 'https://www.sephora.com//product/the-ordinary-deciem-hyaluronic-acid-2-b5-P427419?skuId=2031375&icid2=products'
            params2 = {'api_key':api, 'url':link}
            product_pg = requests.get('http://api.scraperapi.com', params =urlencode(params2), headers=headers)
            product_soup1 = BeautifulSoup(product_pg.content, "html.parser")
            product_soup = BeautifulSoup(product_soup1.prettify(), "html.parser")
            #print(product_soup.prettify())
            #json_dict = json.loads(product_soup.find("script", {"id": "linkStore"}).text)
            #script_tag = product_soup.find("script", {"id": "linkStore"})
            script_tag = product_soup.find("script", {"id": "linkStore", "type": "text/json"})


            if script_tag:
                script_text = script_tag.text

                try:
                    json_dict = json.loads(script_text)
                    # Now you can work with json_dict
                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON: {e}")
            else:
                print("Script tag with id 'linkStore' not found.")

            

            '''hierarchy = get_product_breadcrumbs(json_dict)
            brand = get_brand(json_dict)
            product = get_product_name(json_dict)
            rating = get_rating(json_dict)
            loves_count = get_lovesCount(json_dict)
            reviews_num = get_review_num(json_dict)
            sku = get_sku(json_dict)
            size = get_size(json_dict)
            price = get_price(json_dict)
            child_sku = get_child_sku(json_dict)
            item_id = get_item_id(json_dict)
            similar_products = get_similar_products(product_id = item_id, headers =headers )
            skin_concern = get_skin_concern(json_dict)
            skin_type = get_skin_type(json_dict)

            skincare_dict[sku] = {}
            skincare_dict[sku]['hierarchy'] = ','.join(hierarchy)
            skincare_dict[sku]['brand'] = brand 
            skincare_dict[sku]['product'] = product 
            skincare_dict[sku]['rating'] = rating 
            skincare_dict[sku]['loves_count'] = loves_count 
            skincare_dict[sku]['reviews_num'] = reviews_num
            skincare_dict[sku]['size'] = size 
            skincare_dict[sku]['price'] = price 
            skincare_dict[sku]['child_sku'] = ','.join(child_sku) 
            skincare_dict[sku]['item_id'] = item_id
            skincare_dict[sku]['similar_products'] = ','.join(similar_products)
            skincare_dict[sku]['skin_concern'] = ','.join(skin_concern)
            skincare_dict[sku]['skin_type'] = ','.join(skin_type)'''


    return link_lst, skincare_dict

In [207]:
headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"}
link_list, all_products_dict = get_skincare_info(url= 'https://www.sephora.com/shop/skincare?currentPage=', total_pg =1 , headers = headers, api ='57d761a02ba8a95e8bcad4641873ab9b')


https://www.sephora.com/shop/skincare?currentPage=1
['https://www.sephora.com//product/the-ordinary-deciem-hyaluronic-acid-2-b5-P427419?skuId=2031375&icid2=products grid:p427419:product', 'https://www.sephora.com//product/the-ordinary-deciem-niacinamide-10-zinc-1-P427417?skuId=2031391&icid2=products grid:p427417:product', 'https://www.sephora.com//product/glow-recipe-watermelon-glow-pha-bha-pore-tight-toner-P458219?skuId=2348431&icid2=products grid:p458219:product', 'https://www.sephora.com//product/lip-sleeping-mask-P420652?skuId=1966258&icid2=products grid:p420652:product', 'https://www.sephora.com//product/touchland-power-mist-hydrating-hand-sanitizer-P480529?skuId=2556744&icid2=products grid:p480529:product', 'https://www.sephora.com//product/protini-tm-polypeptide-cream-P427421?skuId=2025633&icid2=products grid:p427421:product', 'https://www.sephora.com//product/glow-recipe-watermelon-glow-niacinamide-dew-drops-P466123?skuId=2404846&icid2=products grid:p466123:product']
first elem

In [210]:
for i in link_list:
    print(link_list)

['https://www.sephora.com//product/the-ordinary-deciem-hyaluronic-acid-2-b5-P427419?skuId=2031375&icid2=products grid:p427419:product', 'https://www.sephora.com//product/the-ordinary-deciem-niacinamide-10-zinc-1-P427417?skuId=2031391&icid2=products grid:p427417:product', 'https://www.sephora.com//product/glow-recipe-watermelon-glow-pha-bha-pore-tight-toner-P458219?skuId=2348431&icid2=products grid:p458219:product', 'https://www.sephora.com//product/lip-sleeping-mask-P420652?skuId=1966258&icid2=products grid:p420652:product', 'https://www.sephora.com//product/touchland-power-mist-hydrating-hand-sanitizer-P480529?skuId=2556744&icid2=products grid:p480529:product', 'https://www.sephora.com//product/protini-tm-polypeptide-cream-P427421?skuId=2025633&icid2=products grid:p427421:product', 'https://www.sephora.com//product/glow-recipe-watermelon-glow-niacinamide-dew-drops-P466123?skuId=2404846&icid2=products grid:p466123:product']
['https://www.sephora.com//product/the-ordinary-deciem-hyaluro

In [158]:
all_products_df = pd.DataFrame.from_dict(all_products_dict).transpose()
all_products_df.to_csv('SephoraSkincareData.csv', sep='\t')

In [163]:
print(len(all_products_dict))

14
